In [1]:
from sympy import symbols, Eq, solve, Matrix
import sympy as sp
import math

## Gregory Maju

In [2]:
data = [
    [1, 3.5, 6, 8.5],  # X
    [2.7, 3.32, 12.18, 20.1]  # Y
]
pembulatan = 5

x = 5
h = data[0][1] - data[0][0]
s = (x - data[0][0])/h
print("======================== COMPONENTS ========================")
print(f"x: {x}")
print(f"h: {h}")
print(f"s: {s:.5f}\n")

n = len(data[1])

# res = [
#     [2.7],
#     [3.32],
#     [12.18],
#     [20.1]
# ]
res = [[y] for y in data[1]]

# ===== Calculation Tables =====
for i in range(n-1):
    for j in range(n-i-1):
        curr = res[j][i]
        curr_forward = res[j+1][i]
        temp = curr_forward - curr
        res[j].append(temp)

print("======================== TABLE ========================")
for val in res:
    for v in val:
        print(f"{round(v, pembulatan):<12}", end="")
    print("")
print("")

# ===== Derivative Calculation =====

# Coefficient for derivative
derivative = res[0][1] / h  # Start with Δy₀ / h (first-order term)

# for i in range(2, len(res[0])):  # Start from the second-order term
#     term = res[0][i]  # Δ^i y₀
#     factorial = math.factorial(i)  # Factorial of the current order
#     # Calculate the coefficient (multiplier for the current term)
#     if i == 2:
#         coefficient = (2 * s - 1)
#     elif i == 3:
#         coefficient = (3 * s**2 - 6 * s + 2)
#     elif i == 4:
#         coefficient = (4 * s**3 - 18 * s**2 + 22 * s - 6)
#     else:
#         raise ValueError("Higher-order terms need to be explicitly defined.")
#     # Add the term to the derivative
#     derivative += (term * coefficient) / (factorial * h)

for i in range(2, len(res[0])):  # Start from the second-order term
    term = res[0][i]  # Δ^i y₀
    factorial = math.factorial(i)  # Factorial of the current order
    # Calculate the coefficient (multiplier for the current term)
    s_ = symbols('s')
    coefficient = s_
    for i in range(1, i):
        coefficient *= (s_ - i)
    coefficient = sp.diff(coefficient, s_)
    coefficient = coefficient.subs({s_: s}).evalf()
    derivative += (term * coefficient) / (factorial * h)


print("======================== RESULT ========================")
print(f"y'({x}) = {round(derivative, pembulatan)}")

======================== COMPONENTS ========================
x: 5
h: 2.5
s: 1.60000

======================== TABLE ========================
2.7         0.62        8.24        -9.18       
3.32        8.86        -0.94       
12.18       7.92        
20.1        

======================== RESULT ========================
y'(5) = 3.82464


## Gregory Mundur

In [43]:
data = [
    [1, 3.5, 6, 8.5],  # X
    [2.7, 3.32, 12.18, 20.1]  # Y
]
pembulatan = 5

x = 5
h = data[0][1] - data[0][0]
s = (x - data[0][-1])/h
print("======================== COMPONENTS ========================")
print(f"x: {x}")
print(f"h: {h}")
print(f"s: {s:.5f}\n")

n = len(data[1])

# res = [
#     [-0.63212],
#     [-0.27763],
#     [0.10653],
#     [0.5288],
#     [1]
# ]
res = [[y] for y in data[1]]

# ===== Calculation Tables =====
for i in range(n-1):
    for j in range(n-i-1):
        curr = res[j][i]
        curr_forward = res[j+1][i]
        temp = curr_forward - curr
        res[j].append(temp)

print("======================== TABLE ========================")
for val in res:
    for v in val:
        print(f"{round(v, pembulatan):<12}", end="")
    print("")
print("")

# ===== Calculation Value =====
res_inv = res[::-1]
y = res_inv[1][-1]
cnt = 2

for val in res[::-1][2:]:
    # val = round(val, 5)
    val = val[-1] / math.factorial(cnt)
    s_sum = 0
    for j in range(cnt):
        s_temp = 1
        for k in range(cnt):
            if k != j:
                s_temp *= (s + k)
        s_sum += s_temp
    val *= s_sum    
    cnt += 1
    y += val
    
y /= h
print("======================== RESULT ========================")
print(f"y'({x}) = {round(y, pembulatan)}")

======================== COMPONENTS ========================
x: 5
h: 2.5
s: -1.40000

======================== TABLE ========================
2.7         0.62        8.24        -9.18       
3.32        8.86        -0.94       
12.18       7.92        
20.1        

======================== RESULT ========================
y'(5) = 3.82464


## Lagrange

In [51]:
data = [
    [1, 3.5, 6, 8.5],  # X
    [2.7, 3.32, 12.18, 20.1]  # Y
]

pembulatan = 5

n = len(data[1])
x = 5

'''
coords = [
    [i, Li, Pi]
]
'''
coords = [
    [0],
    [1],
    [2],
    [3]
]

# ===== CALCULATE Li =====
for idx, coord in enumerate(coords):
    i = coord[0]
    xi = data[0][i]
    li = 1
    numerator = 0
    denominator = 1
    # ===== code here =====
    # for j_idx, coord2 in enumerate(coords):
    #     j = coord2[0]
    #     if j != i:
    #         # j = coords[j][0]
    #         xj = data[0][j]
    #         numerator += (x - xj)
    #         denominator *= (xi - xj)
    #         # li *= ((x - xj) / (xi - xj))
    # li = numerator / denominator
    
    # Numerator
    for j_idx, coord2 in enumerate(coords):
        j = coord2[0]
        j_temp = 0
        if j != i:
            k_temp = 1
            for k_idx, coord3 in enumerate(coords):
                k = coord3[0]
                if k != j and k != i:
                    xk = data[0][k]
                    k_temp *= (x - xk)
            j_temp += k_temp
        numerator += j_temp
        
    # Denominator
    for k_idx, coord2 in enumerate(coords):
        k = coord2[0]
        if k != i:
            xk = data[0][k]
            denominator *= (xi - xk)
    
    li = numerator / denominator
    
    pi = data[1][i] * li
    coords[idx].append(li)
    coords[idx].append(pi)

print("======================== TABLE ==========================")
print(f"{'x':<12}{'f(x)':<12}{'L`i(xi)':<12}{'Pi':<12}")
for coord in coords:
    i = coord[0]
    print(f"{data[0][i]:<12}", end="")
    print(f"{data[1][i]:<12}", end="")
    print(f"{round(coord[1], pembulatan):<12}", end="")
    print(f"{round(coord[2], pembulatan):<12}", end="")
    print("")
print("")

# ===== CALCULATE Pn(x) =====
pn = 0
for coord in coords:
    pn += coord[-1]

print("======================== RESULT ==========================")
print(f"f'({x}) = {round(pn, pembulatan)}")

======================== TABLE ==========================
x           f(x)        L`i(xi)     Pi          
1           2.7         0.03467     0.0936      
3.5         3.32        -0.464      -1.54048    
6           12.18       0.424       5.16432     
8.5         20.1        0.00533     0.1072      

======================== RESULT ==========================
f'(5) = 3.82464
